In [1]:
import itertools
from typing import Optional

import ase
import numpy
from ase.calculators.calculator import Calculator
from sella import Constraints, Internals, Sella


def optimize(
    atms_obj: ase.Atoms,
    calc: Calculator,
    order: int = 0,
    ints_obj: Optional[Internals] = None,
    in_place: bool = False,
) -> ase.Atoms:
    """Optimize a geometry using Sella

    :param atms_obj: An ASE Atoms object
    :param calc: An ASE Calculator object
    :param order: 0 = minimum | 1 = saddle point
    :param ints_obj: A Sella Internals object, possibly involving constraints
    :param in_place: Modify the atoms object in place?
    """
    if not in_place:
        atms_obj = atms_obj.copy()

    atms_obj.calc = calc

    # Initialize and run the optimization
    dyn = Sella(
        atms_obj, order=order, internal=(True if ints_obj is None else ints_obj)
    )
    dyn.run()

    return atms_obj


def optimize_minimum(
    atms_obj: ase.Atoms, calc: Calculator, in_place: bool = False
) -> ase.Atoms:
    """Optimize a minimum-energy structure using Sella

    :param atms_obj: An ASE Atoms object
    :param calc: An ASE Calculator object
    :param in_place: Modify the atoms object in place?
    """
    return optimize(atms_obj, calc, order=0, in_place=in_place)


def optimize_ts(
    atms_obj: ase.Atoms, calc: Calculator, in_place: bool = False
) -> ase.Atoms:
    """Optimize a TS/saddle-point structure using Sella

    :param atms_obj: An ASE Atoms object
    :param calc: An ASE Calculator object
    :param in_place: Modify the atoms object in place?
    """
    return optimize(atms_obj, calc, order=1, in_place=in_place)


def optimize_constrained(
    atms_obj: ase.Atoms,
    calc: Calculator,
    const_coos: Optional[list[tuple[int, ...]]] = None,
    in_place: bool = False,
) -> ase.Atoms:
    """Optimize a structure subject to internal coordinate constraints using Sella

    :param atms_obj: An ASE Atoms object
    :param calc: An ASE Calculator object
    :param const_coos: Optionally, constrain a set of coordinates
    :param in_place: Modify the atoms object in place?
    """
    if not in_place:
        atms_obj = atms_obj.copy()

    # Set up constraints
    const_obj = Constraints(atms_obj)
    for coo in const_coos:
        print("Applying constraint:", coo)
        if len(coo) == 2:
            const_obj.fix_bond(coo)
        elif len(coo) == 3:
            const_obj.fix_angle(coo)
        elif len(coo) == 4:
            const_obj.fix_dihedral(coo)

    # Set up internal coordinates
    ints_obj = Internals(atms_obj, cons=const_obj)
    ints_obj.find_all_bonds()
    ints_obj.find_all_angles()
    ints_obj.find_all_dihedrals()

    # Note: For whatever reason, the Atoms object must be identical to the one passed in
    # to Constraints and Internals, so we *must* set in_place=True here
    return optimize(atms_obj, calc, ints_obj=ints_obj, in_place=True)


def scan_relaxed(
    atms_obj: ase.Atoms,
    calc: Calculator,
    scan_coos: Optional[list[tuple[int, ...]]] = None,
    scan_vals: Optional[list[numpy.ndarray]] = None,
    const_coos: Optional[list[tuple[int, ...]]] = None,
    in_place: bool = False,
) -> list[ase.Atoms]:
    """Perform a relaxed scan with constraints using Sella

    :param atms_obj: An ASE Atoms object
    :param calc: An ASE Calculator object
    :param scan_coos: The set of coordinates to scan over
    :param scan_vals: The set of values to scan over for each coordinate
    :param const_coos: Optionally, constrain a set of coordinates
    :param in_place: Modify the atoms object in place?
    """
    if not in_place:
        atms_obj = atms_obj.copy()

    all_const_coos = list(scan_coos) + list(const_coos)
    const_idxs = sorted(set(itertools.chain(*const_coos)))

    scan_grids = numpy.meshgrid(*scan_vals)

    shape = numpy.shape(scan_grids[0])
    atms_obj_grid = numpy.empty(shape, dtype=numpy.object_)
    ene_grid = numpy.empty(shape, dtype=numpy.float_)

    for idx in numpy.ndindex(shape):
        for coo, grid in zip(scan_coos, scan_grids):
            print(f"Setting {coo} to {grid[idx]}")
            if len(coo) == 2:
                atms_obj.set_distance(*coo, grid[idx], fix=1, indices=const_idxs)
            elif len(coo) == 3:
                atms_obj.set_angle(*coo, grid[idx], fix=1, indices=const_idxs)
            elif len(coo) == 4:
                atms_obj.set_dihedral(*coo, grid[idx], fix=1, indices=const_idxs)

        optimize_constrained(atms_obj, calc, const_coos=all_const_coos, in_place=True)

        atms_obj_grid[idx] = atms_obj.copy()
        ene_grid[idx] = atms_obj.get_total_energy()

    return atms_obj_grid, ene_grid

In [2]:
from tblite.ase import TBLite

import automol

calc = TBLite(method="GFN1-xTB")

rsmi, psmi = ("CCCO[O]", "[CH2]CCOO")
# rsmi, psmi = ("CCC[CH2]", "CC[CH]C")
rsmis = automol.smiles.split(rsmi)
psmis = automol.smiles.split(psmi)

# 1. Find minimum energy geometries for the reactants
ratms_objs = list(map(automol.geom.ase_atoms, map(automol.smiles.geometry, rsmis)))
ratms_objs = [optimize_minimum(o, calc=calc) for o in ratms_objs]
rgeos = list(map(automol.geom.from_ase_atoms, ratms_objs))

# 2. Map the reaction
rxn, *_ = automol.reac.from_smiles(rsmis, psmis, stereo=False)

# 3. Generate a TS guess structure
rxn = automol.reac.with_structures(rxn, "geom", rct_strucs=rgeos)
geo0 = automol.reac.ts_structure(rxn)
scan_coos = automol.reac.scan_coordinates(rxn)
scan_vals = automol.reac.scan_values(rxn)
const_coos = automol.reac.constraint_coordinates(rxn)

# 4. Do a relaxed scan over the reaction coordinate
atms_obj = automol.geom.ase_atoms(geo0)
atms_obj_grid, ene_grid = scan_relaxed(
    atms_obj=atms_obj,
    calc=TBLite(method="GFN1-xTB"),
    scan_coos=scan_coos,
    scan_vals=scan_vals,
    const_coos=const_coos,
)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
/home/avcopan/code/amech-dev/src/autochem/sella-repo/notebook-examples/.pixi/envs/default/lib/python3.12/site-packages/ase/optimize/optimize.py:372: FutureWarning: force_consistent keyword is deprecated and will be ignored.  This will raise an error in future versions of ASE.
  warnings.warn(


------------------------------------------------------------
  cycle        total energy    energy error   density error
------------------------------------------------------------
      1     -19.02862858485  -1.9167817E+01   5.4539771E-01
      2     -19.30535598436  -2.7672740E-01   3.2847714E-01
      3     -19.31963568696  -1.4279703E-02   1.5419884E-01
      4     -19.32161537200  -1.9796850E-03   2.3362542E-02
      5     -19.32284860375  -1.2332318E-03   6.3206218E-03
      6     -19.32299179301  -1.4318926E-04   8.3588138E-04
      7     -19.32299404309  -2.2500827E-06   3.4373169E-04
      8     -19.32299431728  -2.7418129E-07   6.8612568E-05
      9     -19.32299432419  -6.9166894E-09   3.5420840E-05
     10     -19.32299432677  -2.5747937E-09   9.1872741E-06
------------------------------------------------------------

 total:                                   1.049 sec
     Step     Time          Energy         fmax         cmax       rtrust          rho
Sella   0 15:50:3

/home/avcopan/code/amech-dev/src/autochem/sella-repo/notebook-examples/.pixi/envs/default/lib/python3.12/site-packages/ase/optimize/optimize.py:372: FutureWarning: force_consistent keyword is deprecated and will be ignored.  This will raise an error in future versions of ASE.
  warnings.warn(


      1     -18.99058389979  -1.9169926E+01   5.3630701E-01
      2     -19.22818771072  -2.3760381E-01   3.3914319E-01
      3     -19.24006936603  -1.1881655E-02   1.5373694E-01
      4     -19.24885542662  -8.7860606E-03   2.6868159E-02
      5     -19.25066018449  -1.8047579E-03   4.9118025E-03
      6     -19.25067547437  -1.5289882E-05   1.6698658E-03
      7     -19.25067986289  -4.3885198E-06   7.3505027E-04
      8     -19.25068081377  -9.5087978E-07   2.7326318E-04
      9     -19.25068101138  -1.9761037E-07   3.8636243E-05
     10     -19.25068101352  -2.1404780E-09   1.4198364E-05
------------------------------------------------------------

 total:                                   1.075 sec
     Step     Time          Energy         fmax         cmax       rtrust          rho
Sella   0 15:50:44     -523.837712       7.5634       0.0000       0.1000       1.0000
------------------------------------------------------------
  cycle        total energy    energy error   densi

In [3]:
# Visualize the results
def log_geometry_info(geo, scan_coos, const_coos, gra=None):
    """Print and display information about a geometry

    :param geo: An automol geometry data structure
    :param coo: The reaction/scan coordinate
    :param const_coos: Any constrained coordinates
    """
    # Display the geometry
    automol.geom.display(geo, gra=gra)

    # Print distances for the scan coordinates
    print("Constraint coordinates:")
    for coo in scan_coos:
        dist = automol.geom.distance(geo, *coo, angstrom=True)
        print(f" - {coo} distance: {dist}")

    # Print distances for the constraint coordinates
    print("Constraint coordinates:")
    for coo in const_coos:
        dist = automol.geom.distance(geo, *coo, angstrom=True)
        print(f" - {coo} distance: {dist}")


ts_gra = automol.reac.ts_graph(rxn)
automol.graph.display(ts_gra, label=True, exp=True)

print("\nStarting geometry:")
log_geometry_info(geo0, scan_coos, const_coos, gra=ts_gra)

print("\nScan geometries:")
for idx in numpy.ndindex(numpy.shape(ene_grid)):
    ene_val = ene_grid[idx]
    atms_obj = atms_obj_grid[idx]
    geo = automol.geom.from_ase_atoms(atms_obj)
    log_geometry_info(geo, scan_coos, const_coos, gra=ts_gra)
    print("Energy:", ene_val)
    print("*** ***\n")


Starting geometry:


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Constraint coordinates:
 - (4, 5) distance: 1.8261871272156087
Constraint coordinates:
 - (3, 4) distance: 1.4342032069537345

Scan geometries:


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Constraint coordinates:
 - (4, 5) distance: 1.8261880672570623
Constraint coordinates:
 - (3, 4) distance: 1.4342034252287101
Energy: -525.7753824722134
*** ***



...

Constraint coordinates:
 - (4, 5) distance: 1.601187653001749
Constraint coordinates:
 - (3, 4) distance: 1.4567395070789015
Energy: -525.6946466463459
*** ***



3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Constraint coordinates:
 - (4, 5) distance: 1.563680419845151
Constraint coordinates:
 - (3, 4) distance: 1.4604660779434853
Energy: -525.685170737008
*** ***



3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Constraint coordinates:
 - (4, 5) distance: 1.5261882330119907
Constraint coordinates:
 - (3, 4) distance: 1.464373599542519
Energy: -525.6754737770739
*** ***

